In [ ]:
import os
import shutil


%cd /kaggle/working


repo_name = "benchmarking-generative-models-for-domain-adaptation"
if os.path.exists(repo_name):
    shutil.rmtree(repo_name)


print("Clono la repository...")
!git clone --recursive https://github.com/AntonioRosano/benchmarking-generative-models-for-domain-adaptation.git
%cd {repo_name}


In [ ]:
%cd /kaggle/working/benchmarking-generative-models-for-domain-adaptation/models/pspnet

os.makedirs('initmodel', exist_ok=True)


source_path = "/kaggle/input/resnet50-weights/resnet50_v2.pth" 
destination_path = "./initmodel/resnet50_v2.pth"


if os.path.exists(source_path):
    shutil.copy(source_path, destination_path)
    print(f"Pesi copiati con successo in: {destination_path}")
else:
    print(f"Errore: Non trovo il file in {source_path}")
    print("File disponibili in input:")
    for root, dirs, files in os.walk("/kaggle/input"):
        for file in files:
            print(os.path.join(root, file))

In [ ]:
import random


dataset_root = "/kaggle/input/datasets/antoniorosano/ego-ch/EGO-CH-OBJ-SEG/synthetic/train"
img_dir = os.path.join(dataset_root, "frames")
mask_dir = os.path.join(dataset_root, "labels")

output_folder = "/kaggle/working/benchmarking-generative-models-for-domain-adaptation/models/pspnet/lists/ego_ch"
os.makedirs(output_folder, exist_ok=True)

print(f"Scansiono le cartelle:\n   IMG: {img_dir}\n   MSK: {mask_dir}")


images_dict = {
    os.path.splitext(f)[0]: os.path.join(img_dir, f) 
    for f in os.listdir(img_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))
}

masks_dict = {
    os.path.splitext(f)[0]: os.path.join(mask_dir, f) 
    for f in os.listdir(mask_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))
}

print(f"\nStatistiche grezze:")
print(f"   Immagini trovate: {len(images_dict)}")
print(f"   Labels trovate:   {len(masks_dict)}")

# --- RISOLUZIONE BUG (INTERSEZIONE) ---
# Troviamo solo i nomi presenti in ENTRAMBE le liste
common_names = list(set(images_dict.keys()) & set(masks_dict.keys()))
print(f"\nCoppie valide (MATCH): {len(common_names)}")
print(f"File scartati (senza coppia): {len(images_dict) + len(masks_dict) - 2*len(common_names)}")

# --- SPLIT TRAIN / VAL ---
# Mescoliamo per evitare bias (es. se le immagini sono ordinate)
random.seed(42)
random.shuffle(common_names)

# Usiamo il 90% per training e il 10% per validation
split_idx = int(len(common_names) * 0.9)
train_names = common_names[:split_idx]
val_names = common_names[split_idx:]


def write_list(names, filename):
    path = os.path.join(output_folder, filename)
    with open(path, "w") as f:
        for name in names:
            f.write(f"{images_dict[name]} {masks_dict[name]}\n")
    print(f"Creato: {path} ({len(names)} righe)")

write_list(train_names, "train.txt")
write_list(val_names, "val.txt")

In [ ]:
%cd /kaggle/working/benchmarking-generative-models-for-domain-adaptation/models/pspnet


config_dir = "config/ego_ch"
os.makedirs(config_dir, exist_ok=True)

# YAML parametri del PAPER (LR=0.005, Epochs=30, ecc.)
config_content = """
DATA:
  data_root: /
  train_list: /kaggle/working/benchmarking-generative-models-for-domain-adaptation/models/pspnet/lists/ego_ch/train.txt
  val_list: /kaggle/working/benchmarking-generative-models-for-domain-adaptation/models/pspnet/lists/ego_ch/val.txt
  classes: 25  # 24 opere + 1 sfondo (Verifica se le tue label sono 0-24 o 0-23)

TRAIN:
  arch: psp
  layers: 50
  sync_bn: False       
  train_h: 473         # Crop
  train_w: 473
  scale_min: 0.5
  scale_max: 2.0
  rotate_min: -10
  rotate_max: 10
  zoom_factor: 8
  ignore_label: 255
  aux_weight: 0.4
  train_gpu: [0]
  workers: 2           
  batch_size: 4
  batch_size_val: 4
  base_lr: 0.005       # Parametro dal PAPER
  epochs: 30           # Parametro dal PAPER
  start_epoch: 0
  power: 0.9           # Parametro dal PAPER
  momentum: 0.9        # Parametro dal PAPER
  weight_decay: 0.0001 # Parametro dal PAPER
  manual_seed: 0
  print_freq: 10
  save_freq: 1
  save_path: exp/ego_ch/pspnet50/model
  weight: initmodel/resnet50_v2.pth  # Il file che abbiamo sistemato prima
  resume: 
  evaluate: True

Distributed:
  dist_url: tcp://127.0.0.1:6789
  dist_backend: 'nccl'
  multiprocessing_distributed: False
  world_size: 1
  rank: 0
  use_apex: False
  opt_level: 'O0'
  keep_batchnorm_fp32: None
  loss_scale: None
"""


config_path = os.path.join(config_dir, "pspnet50.yaml")
with open(config_path, "w") as f:
    f.write(config_content)

print(f"Configurazione creata in: {config_path}")

In [ ]:
import os

print("Risoluzione bug PSPNet noti...")

# --- 1. PATCH TRAIN.PY (Gestione Pesi) ---
train_path = "/kaggle/working/benchmarking-generative-models-for-domain-adaptation/models/pspnet/tool/train.py"
if os.path.exists(train_path):
    with open(train_path, 'r') as f:
        lines = f.readlines()
    new_lines = []
    target = "model.load_state_dict(checkpoint['state_dict'])"
    for line in lines:
        if target in line:
            indent = line[:line.find("model")]
            new_lines.extend([
                f"{indent}# --- PATCH APPLICATA: Supporto per pesi raw ---\n",
                f"{indent}if isinstance(checkpoint, dict) and 'state_dict' in checkpoint:\n",
                f"{indent}    model.load_state_dict(checkpoint['state_dict'], strict=False)\n",
                f"{indent}else:\n",
                f"{indent}    print('=> Key state_dict not found, loading raw weights...')\n",
                f"{indent}    model.load_state_dict(checkpoint, strict=False)\n"
            ])
        else:
            new_lines.append(line)
    with open(train_path, 'w') as f:
        f.writelines(new_lines)
    print("Patch train.py applicata.")

# --- 2. PATCH TRANSFORM.PY (Compatibilità Python 3.10+) ---
trans_path = "/kaggle/working/benchmarking-generative-models-for-domain-adaptation/models/pspnet/util/transform.py"
if os.path.exists(trans_path):
    with open(trans_path, "r") as f:
        content = f.read()
    if "collections.Iterable" in content:
        with open(trans_path, "w") as f:
            f.write(content.replace("collections.Iterable", "collections.abc.Iterable"))
        print("Patch transform.py applicata (Iterable fix).")

# --- 3. REWRITE DATASET.PY (Sanitizzazione Maschere) ---
dataset_code = """
import os
import os.path
import cv2
import numpy as np
from torch.utils.data import Dataset

def make_dataset(split='train', data_root=None, data_list=None):
    image_label_list = []
    list_read = open(data_list).readlines()
    for line in list_read:
        line = line.strip().split(' ')
        if split == 'test':
            image_label_list.append(os.path.join(data_root, line[0]))
        else:
            image_label_list.append((os.path.join(data_root, line[0]), os.path.join(data_root, line[1])))
    return image_label_list

class SemData(Dataset):
    def __init__(self, split='train', data_root=None, data_list=None, transform=None):
        self.split = split
        self.data_list = make_dataset(split, data_root, data_list)
        self.transform = transform

    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, index):
        image_path, label_path = self.data_list[index]
        image = cv2.imread(image_path, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = np.float32(image)
        # Forza grayscale e sanitizza per evitare crash CUDA
        label = cv2.imread(label_path, cv2.IMREAD_GRAYSCALE)
        if label.ndim == 3: label = label[:, :, 0]
        # Tutto ciò che eccede le classi (es. 25) diventa 255 (ignore)
        bad_pixels = (label >= 25) & (label != 255)
        if np.any(bad_pixels): label[bad_pixels] = 255
        
        if self.transform is not None:
            image, label = self.transform(image, label)
        return image, label
"""
dataset_path = "/kaggle/working/benchmarking-generative-models-for-domain-adaptation/models/pspnet/util/dataset.py"
with open(dataset_path, "w") as f:
    f.write(dataset_code)
print("Patch dataset.py applicata (Mask sanitizer).")

print("\nTUTTE LE FIX SONO STATE APPLICATE. Il sistema è pronto per il training.")

In [ ]:
%cd /kaggle/working/benchmarking-generative-models-for-domain-adaptation/models/pspnet

import sys
import os
os.environ['PYTHONPATH'] = os.getcwd()

!pip install tensorboardX
!python tool/train.py --config config/ego_ch/pspnet50.yaml